In [18]:
import os 
import zipfile
import pandas as pd
import numpy as np

import bbg_symbology as bbs
from bbg_loader_core import *

In [3]:
def merge_dict(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

class ExpiredAliasService:
    def __init__(self, src=None, dbfldr=None):
        
        self._src = src
        self._dbfldr = dbfldr
        
        if src is not None:
            self.expired = list(set(pd.read_csv(src, header=None)[1]))
            
            if self._dbfldr is not None:
                dblist = self.get_db_list()
                self.expired = [e for e in self.expired if e in dblist]
        else:
            self.expired = []
    
    def get_db_list(self):
        extension = '.pickle'
        return [f.replace(extension, '') for f in os.listdir(self._dbfldr) if f.endswith(extension)]
    
    def __contains__(self, key):
        return key in self.expired
    
    def __repr__(self):
        return '<{}: {}>'.format(self.__class__, str(self.expired))
    
    def add_expired(self, aliases):
        if isinstance(aliases, list):
            self.expired = self.expired + aliases
        else:
            self.expired.extend([aliases])
        
    def save(self, fname=None):
        if fname is None:
            fname = self._src
            
        if fname is None:
            raise Error('No Filename Specified!')
            
        pd.Series(self.expired).to_csv(fname)
    

In [4]:
def generate_futures_list(out_path):
    
    # init - flds & default params
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['LAST_TRADEABLE_DT']

    default_params = {'local_path':  out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    
    # get all unique roots (alias roots)
    roots = list(FutAliasRef._fut_ref.Alias)
    
    # store outputs
    fut_list = {}
    
    # loop through roots
    for root in roots:
        
        # get all bbg, alias tckrs
        bbg_tckrs  = FutChainRef.get_futures_chain(root)
        alias_tckrs = [FutAliasRef.bbg_to_alias_tckr(tckr) for tckr in bbg_tckrs]
        tckr_list = zip(bbg_tckrs, alias_tckrs)
        
        # loop through ticker combos... and return object
        for bbg, alias in tckr_list:
            if alias not in ExpiredFutures:
                p = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
                fut_list[alias] = BbgSecurity(**p)
                
    return fut_list


def generate_bbg_list(src_file, out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):
    tckrList = pd.read_csv(src_file)
    default_params = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}

    secList = {}    
    for (i, (bbg,alias)) in tckrList.iterrows():
        d = merge_dict({'bb_tckr': bbg, 'alias': alias}, default_params)
        secList[bbg] = BbgSecurity(**d)

    return secList


def generate_generic_fut_list(out_path, ts_flds=['px_last'], meta_flds=['NAME','LONG_COMP_NAME']):

    def gen_obj(broot, rmtd, yk, ng, alias_root, alias_suff, dparams):
        bbg_tckr   = broot.upper()+ str(ng) + ' ' + rmtd + ' ' + yk
        alias_tckr = alias_root + str(ng) + '.' + alias_suff

        d = merge_dict({'bb_tckr': bbg_tckr, 'alias': alias_tckr}, dparams)
        return alias_tckr, BbgSecurity(**d)
    
    ts_flds =   ['px_last', 'open_int', 'volume']
    meta_flds = ['NAME', 'PX_POS_MULT_FACTOR', 'EXCH_CODE', 'FUT_EXCH_NAME_LONG', 
                 'CRNCY', 'FUT_TICK_SIZE', 'FUT_GEN_MONTH']

    default_params_ridx = {'local_path': out_path, 'ts_flds': ts_flds, 'meta_flds': meta_flds}
    default_params_px   = {'local_path': out_path, 'ts_flds': ts_flds[:1], 'meta_flds': meta_flds[:1]}
    
    
    n = 1
    secList = {}    
    for idx, (root, num_gen, yk, alias) in FutAliasRef._fut_ref.iterrows():
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, 1, alias, 'bbd', default_params_ridx)
        secList[atckr] = bbobj
        
        # Price 
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, 1, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj
        
        # Return Index
        atckr, bbobj = gen_obj(root, 'B:00_0_D', yk, num_gen, alias, 'bbd', default_params_px)
        secList[atckr] = bbobj
        
        # Price
        atckr, bbobj = gen_obj(root, 'B:00_0_N', yk, num_gen, alias, 'bbd.px', default_params_px)
        secList[atckr] = bbobj

    return secList

def update_bbg_securities(bbglist):
    expired = []
    error_list = []
    print('--------------UPDATING {}----------------'.format(db))
    for i, (alias, bbgObj) in enumerate(bbglist.items()):
        print('\n(' + str(i) + ') ' + str(bbgObj))

        try:
            bbgObj.update()
            if bbgObj.is_expired:
                expired.append(bbgObj.alias)
        except:
            try: 
                print('...Error with {}, attempting to load from scratch'.format(bbgObj.bb_tckr))
                bbgObj.load_from_scratch()
                bbgObj.save()
                print('...success!')

                if bbgObj.is_expired:
                    expired.append(bbgObj.alias)
            except:
                print('FATAL ERROR WITH {}'.format(bbgObj.bb_tckr))
                error_list.extend(bbgObj)
    
    return expired, error_list

In [20]:
def alias_to_date(alias):
    dtinfo = alias.split('.')[1]
    month_str = dtinfo[0]
    yr_str = dtinfo[1:]
    
    MONTH_REF = 'FGHJKMNQUVXZ'
    month =  MONTH_REF.find(month_str) + 1
    
    yr =  int(yr_str)
    if yr > 50:
        yr = yr + 1900
    else:
        yr = yr + 2000

    return pd.datetime(yr, month, 1)

def get_sec_health(bbg):
    #bbg.load_local_data()
    health = bbg.ts.count()
    
    if 'LAST_TRADEABLE_DT' in bbg.meta:
        health['exp_error'] = np.abs((bbg.meta.LAST_TRADEABLE_DT - alias_to_date(bbg.alias)).days)
    
    try: 
        health['first_dt'] = (bbg.ts.first_valid_index()).strftime('%Y-%m-%d')
    except: 
        health['first_dt'] = None
        
    try: 
        health['last_dt'] = (bbg.ts.last_valid_index()).strftime('%Y-%m-%d')
    except:
        health['last_dt'] = None
        
    health.name = bbg.alias
    return health

def filter_health(df):

    err = pd.DataFrame()
    
    ts_flds = ['px_last', 'open_int', 'volume']

    for fld in ts_flds:
        if fld in df:
            err[fld] = df.px_last==0

    if 'exp_error' in df:
        err['exp'] = df.exp_error > 50

    return df[err.any(axis=1)]

# TODO THIS FUNCTION NEEDS WORK
def export_db_health(dbname, dbfldr='../_bbgDB/'):
        DIR = dbfldr + dbname + '/'

        flist = [f for f in os.listdir(DIR) if f.endswith('.pickle')]
        health_list = []
        
        for f in flist[:100]:
            print(f)
            bbg = BbgSecurity.from_file(DIR+f)
            health_list.append(get_sec_health(bbg))

        return health_list
    
        healthdf = pd.DataFrame(health_list)
        healthdf.update({dbname:pd.DataFrame(health_list)})
        filter_health(healthdf).to_excel('../_bbgDB/health_' + dbname + '.xlsx')
        

def zip_folder(fldr, zip_dest_fldr, zip_filename):
    zf = zipfile.ZipFile(zipfldr + zipfilename, 'w', zipfile.ZIP_DEFLATED)
    for dirname, subdirs, files in os.walk(fldr):    
        zf.write(dirname)
        for filename in files:
            zf.write(os.path.join(dirname, filename))
    zf.close()


In [5]:
INPUT_PATH = '../_in/'

FUTURES_MONTHS = {m:i+1 for i,m in enumerate('FGHJKMNQUVXZ')}

FutChainRef    = bbs.FuturesChainReference(INPUT_PATH + 'futures_historical_chain.csv')
FutAliasRef    = bbs.FuturesAliasService(INPUT_PATH + 'fut_roots.csv')
BbgTckrService = bbs.BloombergTckrService()
ExpiredFutures = ExpiredAliasService('../_in/expired_aliases.csv', '../_bbgDB/Futures/')


In [6]:
BBG_UPDATE_LIST = {'CoT':           generate_bbg_list(src_file=INPUT_PATH + 'CoT.csv',           out_path='../_bbgDB/CoT/'),
                   'Index':         generate_bbg_list(src_file=INPUT_PATH + 'Index.csv',         out_path='../_bbgDB/Index/'),
                   'FX':            generate_bbg_list(src_file=INPUT_PATH + 'FX.csv',            out_path='../_bbgDB/FX/'), 
                   'InterestRates': generate_bbg_list(src_file=INPUT_PATH + 'InterestRates.csv', out_path='../_bbgDB/InterestRates/'),
                   'Futures_gen':   generate_generic_fut_list(                                   out_path='../_bbgDB/Futures_gen/'),
                   'Futures':       generate_futures_list(                                       out_path='../_bbgDB/Futures/')}

# print # securities in each
for name, db in BBG_UPDATE_LIST.items():
    print(name, len(db))


('Index', 4)
('CoT', 14)
('FX', 68)
('Futures', 2093)
('Futures_gen', 230)
('InterestRates', 46)


In [8]:
tckrs = list(BBG_UPDATE_LIST['Futures'].keys())
tckrs_ = [tckr for tckr in tckrs if tckr.split('.')[0]=='ng']

In [9]:
tckr = 'gc.N84'
bb = BbgSecurity.from_file('../_bbgDB/Futures/' + tckr + '.pickle')
bb.is_expired

True

In [10]:
expired = {}
error_list = {}

for db, bbglist in BBG_UPDATE_LIST.items():
    expired[db], error_list[db] = update_bbg_securities(bbglist)

--------------UPDATING Index----------------

(0) <BBG SECURITY: tckr=SPGSCIP Index, alias=gsci.er, local_file=../_bbgDB/Index/>
...updating security SPGSCIP Index
...loading local file <../_bbgDB/Index/gsci.er.pickle> for security=SPGSCIP Index
...updating timeseries for SPGSCIP Index --> loading data from 09/19/2019 to 09/30/2019
...saving SPGSCIP Index to local file <../_bbgDB/Index/gsci.er.pickle>

(1) <BBG SECURITY: tckr=SPGSCITR Index, alias=gsci.tr, local_file=../_bbgDB/Index/>
...updating security SPGSCITR Index
...loading local file <../_bbgDB/Index/gsci.tr.pickle> for security=SPGSCITR Index
...updating timeseries for SPGSCITR Index --> loading data from 09/19/2019 to 09/30/2019
...saving SPGSCITR Index to local file <../_bbgDB/Index/gsci.tr.pickle>

(2) <BBG SECURITY: tckr=NEIXSTTI Index, alias=sgstti, local_file=../_bbgDB/Index/>
...updating security NEIXSTTI Index
...loading local file <../_bbgDB/Index/sgstti.pickle> for security=NEIXSTTI Index
...updating timeseries for N

...saving BPPPCPEU Index to local file <../_bbgDB/FX/eur.cppp.pickle>

(4) <BBG SECURITY: tckr=JPY3M Curncy, alias=jpy.3m, local_file=../_bbgDB/FX/>
...updating security JPY3M Curncy
...loading local file <../_bbgDB/FX/jpy.3m.pickle> for security=JPY3M Curncy
...updating timeseries for JPY3M Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving JPY3M Curncy to local file <../_bbgDB/FX/jpy.3m.pickle>

(5) <BBG SECURITY: tckr=HKD1M Curncy, alias=hkd.1m, local_file=../_bbgDB/FX/>
...updating security HKD1M Curncy
...loading local file <../_bbgDB/FX/hkd.1m.pickle> for security=HKD1M Curncy
...updating timeseries for HKD1M Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving HKD1M Curncy to local file <../_bbgDB/FX/hkd.1m.pickle>

(6) <BBG SECURITY: tckr=EUR3M Curncy, alias=eur.3m, local_file=../_bbgDB/FX/>
...updating security EUR3M Curncy
...loading local file <../_bbgDB/FX/eur.3m.pickle> for security=EUR3M Curncy
...updating timeseries for EUR3M Curncy --> loading da

...saving BPPPCPNO Index to local file <../_bbgDB/FX/nok.cppp.pickle>

(28) <BBG SECURITY: tckr=NZD1M Curncy, alias=nzd.1m, local_file=../_bbgDB/FX/>
...updating security NZD1M Curncy
...loading local file <../_bbgDB/FX/nzd.1m.pickle> for security=NZD1M Curncy
...updating timeseries for NZD1M Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving NZD1M Curncy to local file <../_bbgDB/FX/nzd.1m.pickle>

(29) <BBG SECURITY: tckr=SEK3M Curncy, alias=sek.3m, local_file=../_bbgDB/FX/>
...updating security SEK3M Curncy
...loading local file <../_bbgDB/FX/sek.3m.pickle> for security=SEK3M Curncy
...updating timeseries for SEK3M Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving SEK3M Curncy to local file <../_bbgDB/FX/sek.3m.pickle>

(30) <BBG SECURITY: tckr=BPPPPPCH Index, alias=chf.pppp, local_file=../_bbgDB/FX/>
...updating security BPPPPPCH Index
...loading local file <../_bbgDB/FX/chf.pppp.pickle> for security=BPPPPPCH Index
...updating timeseries for BPPPPPCH Index

...saving NOK3M Curncy to local file <../_bbgDB/FX/nok.3m.pickle>

(52) <BBG SECURITY: tckr=AUD Curncy, alias=aud.spot, local_file=../_bbgDB/FX/>
...updating security AUD Curncy
...loading local file <../_bbgDB/FX/aud.spot.pickle> for security=AUD Curncy
...updating timeseries for AUD Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving AUD Curncy to local file <../_bbgDB/FX/aud.spot.pickle>

(53) <BBG SECURITY: tckr=NOK1M Curncy, alias=nok.1m, local_file=../_bbgDB/FX/>
...updating security NOK1M Curncy
...loading local file <../_bbgDB/FX/nok.1m.pickle> for security=NOK1M Curncy
...updating timeseries for NOK1M Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving NOK1M Curncy to local file <../_bbgDB/FX/nok.1m.pickle>

(54) <BBG SECURITY: tckr=EUR6M  Curncy, alias=eur.6m, local_file=../_bbgDB/FX/>
...updating security EUR6M  Curncy
...loading local file <../_bbgDB/FX/eur.6m.pickle> for security=EUR6M  Curncy
...updating timeseries for EUR6M  Curncy --> loading dat

C:\Anaconda\envs\python2\lib\site-packages\tia\bbg\v3api.py:409: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  frame = frame.reindex_axis(self.request.fields, axis=1)


...

...saving HO1 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/ho1.bbd.px.pickle>

(172) <BBG SECURITY: tckr=DX1 B:00_0_N Curncy, alias=dx1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security DX1 B:00_0_N Curncy
...loading local file <../_bbgDB/Futures_gen/dx1.bbd.px.pickle> for security=DX1 B:00_0_N Curncy
...updating timeseries for DX1 B:00_0_N Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving DX1 B:00_0_N Curncy to local file <../_bbgDB/Futures_gen/dx1.bbd.px.pickle>

(173) <BBG SECURITY: tckr=VG1 B:00_0_N Index, alias=vg1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security VG1 B:00_0_N Index
...loading local file <../_bbgDB/Futures_gen/vg1.bbd.px.pickle> for security=VG1 B:00_0_N Index
...updating timeseries for VG1 B:00_0_N Index --> loading data from 09/20/2019 to 09/30/2019
...saving VG1 B:00_0_N Index to local file <../_bbgDB/Futures_gen/vg1.bbd.px.pickle>

(174) <BBG SECURITY: tckr=FC1 B:00_0_N Comdty, alias=fc1.bbd.px, local_file=../_

...saving CT6 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/ct6.bbd.pickle>

(193) <BBG SECURITY: tckr=SE1 B:00_0_N Curncy, alias=se1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security SE1 B:00_0_N Curncy
...loading local file <../_bbgDB/Futures_gen/se1.bbd.px.pickle> for security=SE1 B:00_0_N Curncy
...updating timeseries for SE1 B:00_0_N Curncy --> loading data from 09/20/2019 to 09/30/2019
...saving SE1 B:00_0_N Curncy to local file <../_bbgDB/Futures_gen/se1.bbd.px.pickle>

(194) <BBG SECURITY: tckr=LH9 B:00_0_D Comdty, alias=lh9.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security LH9 B:00_0_D Comdty
...loading local file <../_bbgDB/Futures_gen/lh9.bbd.pickle> for security=LH9 B:00_0_D Comdty
...updating timeseries for LH9 B:00_0_D Comdty --> loading data from 09/20/2019 to 09/30/2019
...saving LH9 B:00_0_D Comdty to local file <../_bbgDB/Futures_gen/lh9.bbd.pickle>

(195) <BBG SECURITY: tckr=SB5 B:00_0_D Comdty, alias=sb5.bbd, local_file=../_bbgDB/Futu

...updating timeseries for SI6 B:00_0_N Comdty --> loading data from 09/20/2019 to 09/30/2019
...saving SI6 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/si6.bbd.px.pickle>

(214) <BBG SECURITY: tckr=XM1 B:00_0_N Comdty, alias=xm1.bbd.px, local_file=../_bbgDB/Futures_gen/>
...updating security XM1 B:00_0_N Comdty
...loading local file <../_bbgDB/Futures_gen/xm1.bbd.px.pickle> for security=XM1 B:00_0_N Comdty
...updating timeseries for XM1 B:00_0_N Comdty --> loading data from 09/20/2019 to 09/30/2019
...saving XM1 B:00_0_N Comdty to local file <../_bbgDB/Futures_gen/xm1.bbd.px.pickle>

(215) <BBG SECURITY: tckr=UX6 B:00_0_D Index, alias=ux6.bbd, local_file=../_bbgDB/Futures_gen/>
...updating security UX6 B:00_0_D Index
...loading local file <../_bbgDB/Futures_gen/ux6.bbd.pickle> for security=UX6 B:00_0_D Index
...updating timeseries for UX6 B:00_0_D Index --> loading data from 09/20/2019 to 09/30/2019
...saving UX6 B:00_0_D Index to local file <../_bbgDB/Futures_gen/ux6.bbd.pick


(4) <BBG SECURITY: tckr=UKBRBASE Index, alias=uk.cb, local_file=../_bbgDB/InterestRates/>
...updating security UKBRBASE Index
...loading local file <../_bbgDB/InterestRates/uk.cb.pickle> for security=UKBRBASE Index
...updating timeseries for UKBRBASE Index --> loading data from 09/20/2019 to 09/30/2019
...saving UKBRBASE Index to local file <../_bbgDB/InterestRates/uk.cb.pickle>

(5) <BBG SECURITY: tckr=GACGB2 Index, alias=au.2y, local_file=../_bbgDB/InterestRates/>
...updating security GACGB2 Index
...loading local file <../_bbgDB/InterestRates/au.2y.pickle> for security=GACGB2 Index
...updating timeseries for GACGB2 Index --> loading data from 09/20/2019 to 09/30/2019
...saving GACGB2 Index to local file <../_bbgDB/InterestRates/au.2y.pickle>

(6) <BBG SECURITY: tckr=GNOR10YR Index, alias=no.10y, local_file=../_bbgDB/InterestRates/>
...updating security GNOR10YR Index
...loading local file <../_bbgDB/InterestRates/no.10y.pickle> for security=GNOR10YR Index
...updating timeseries for

...saving RBATCTR Index to local file <../_bbgDB/InterestRates/au.cb.pickle>

(27) <BBG SECURITY: tckr=GNOR2YR Index, alias=no.2y, local_file=../_bbgDB/InterestRates/>
...updating security GNOR2YR Index
...loading local file <../_bbgDB/InterestRates/no.2y.pickle> for security=GNOR2YR Index
...updating timeseries for GNOR2YR Index --> loading data from 09/20/2019 to 09/30/2019
...saving GNOR2YR Index to local file <../_bbgDB/InterestRates/no.2y.pickle>

(28) <BBG SECURITY: tckr=GJGB10 Index Index, alias=jp.10y, local_file=../_bbgDB/InterestRates/>
...updating security GJGB10 Index Index
...loading local file <../_bbgDB/InterestRates/jp.10y.pickle> for security=GJGB10 Index Index
...updating timeseries for GJGB10 Index Index --> loading data from 09/20/2019 to 09/30/2019
...saving GJGB10 Index Index to local file <../_bbgDB/InterestRates/jp.10y.pickle>

(29) <BBG SECURITY: tckr=FDTR Index, alias=us.cb, local_file=../_bbgDB/InterestRates/>
...updating security FDTR Index
...loading local 

In [11]:
ExpiredFutures.add_expired(expired['Futures'])
ExpiredFutures.save()

In [17]:
# Database Health Files
databases = ['Futures', 'Futures_gen', 'FX', 'CoT', 'Index', 'InterestRates' ]
health = {}

for db in databases:
    health[db] = export_db_health(db)


In [21]:
# Zip Database into single file for export
fldr = '../_bbgDB'
zipfldr = '../_out/'
zipfilename = 'bbgDB-' + datetime.now().strftime('%Y-%m-%d') + '.zip'

zip_folder(fldr, zipfldr, zipfilename)